In [ ]:
import pyvista as pv
import os

In [ ]:
MESH_Z_SCALE = 0.01
PATH = "2022_11_30_11_52_49_mesh_geometry.vtk"
OUT_PATH = f"{os.path.splitext(os.path.basename(PATH))[0]}.msh"

## Load source mesh

In [ ]:
mesh = pv.read(PATH)
mesh.clear_data()  # clears any field data
mesh

In [ ]:
# Extract surface geometry
surface = mesh.extract_surface()

# Make sure we didn't loose cells
assert mesh.n_cells == surface.n_cells

surface

### Display source mesh

In [ ]:
pl = pv.Plotter()
pl.add_mesh(surface, show_edges=True)
pl.set_scale(zscale=MESH_Z_SCALE)
pl.show()

## Subdivision

Use PyVista's subdivision filter: https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyDataFilters.subdivide.html#subdivide

In [ ]:
# Set the number of subdivisions
N_SUB = 3

In [ ]:
sub = surface.subdivide(
    nsub=N_SUB,  # Number of subdivisions
    subfilter="linear",  # Subdividing algorithm
    progress_bar=True,  # Monitor progress for larger meshes
)
sub

In [ ]:
pl = pv.Plotter()
pl.add_mesh(sub, show_edges=True)
pl.set_scale(zscale=MESH_Z_SCALE)
pl.show()

## Save out to `.msh`

In [ ]:
pv.save_meshio(OUT_PATH, sub)
OUT_PATH

## Comparison of Subdivisions

In [ ]:
pl = pv.Plotter(shape=(2, 3))
for i, method in enumerate(["linear", "butterfly", "loop"]):
    submethod = surface.subdivide(
        nsub=N_SUB,  # Number of subdivisions
        subfilter=method,  # Subdividing algorithm
        progress_bar=True,  # Monitor progress for larger meshes
    )
    # Compute a cell quality metric to make visual comparison easier
    qual = submethod.compute_cell_quality("area")

    pl.subplot(0, i)
    pl.add_text(method)
    pl.add_mesh(qual, show_edges=False, show_scalar_bar=False)
    pl.set_scale(zscale=MESH_Z_SCALE)

    pl.subplot(1, i)
    pl.add_mesh(submethod, show_edges=True)
    pl.set_scale(zscale=MESH_Z_SCALE)


pl.link_views()
pl.show()